# Convolutional Neural Networks Explained

## Imports

In [1]:
import cv2
import scipy.misc
import matplotlib.pyplot as plt
%matplotlib inline

## Load an Image

In [24]:
import os
os.path.isfile('Images/car.png')

False

In [19]:
image_path = 'Images/car.png'
color_image = cv2.imread(image_path)
gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
small_image = scipy.misc.imresize(gray_image, 0.5)
normalize_image = small_image.astype("float32")/255

plt.imshow(normalize_image, cmap='gray')
plt.show()

AttributeError: 'NoneType' object has no attribute 'astype'